In [35]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('stats.csv')

# Convert numeric columns to appropriate types
numeric_cols = ['hit', 'strikeout', 'walk', 'p_out', 'p_win', 'p_earned_run', 
                'p_save', 'p_balk', 'p_quality_start', 'p_hit_by_pitch', 
                'p_hold', 'p_total_pickoff']

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Calculate points using the old scoring system
df['old'] = (
    df['p_win'] * 8 +
    df['p_save'] * 8 +
    df['p_out'] * 1 +
    df['hit'] * -1.3 +
    df['p_earned_run'] * -3 +
    df['walk'] * -1.3 +
    df['p_hit_by_pitch'] * -1.3 +
    df['strikeout'] * 3
)

# Calculate points using the new scoring system (old + additional stats)
df['new'] = (
    df['old'] +
    df['p_balk'] * -1 +
    df['p_hold'] * 4 +
    df['p_total_pickoff'] * 1 +
    df['p_quality_start'] * 3 +
    df['p_save'] * 5 + # double points for saves
    df['p_blown_save'] * -5 # minus points for blown saves
)

# Add rank columns
df['old_rank'] = df['old'].rank(ascending=False, method='min').astype(int)
df['new_rank'] = df['new'].rank(ascending=False, method='min').astype(int)
df['rank_improvement'] = df['old_rank'] - df['new_rank']

# Display the top 20 players with their ranks
top_players = df[['last_name, first_name', 'player_id', 'old', 'old_rank', 'new', 'new_rank', 'p_save']]
top_players = top_players.sort_values(by='new', ascending=False)

print("Top 100 players by new scoring system with ranks:")
with pd.option_context('display.max_rows', None):
    display(top_players.head(100))

# List of the 20 specific players you provided
specific_players = [
    "Helsley, Ryan", "Clase, Emmanuel", "Finnegan, Kyle", "Suarez, Robert", 
    "Hader, Josh", "Iglesias, Raisel", "Yates, Kirby", "Holmes, Clay", 
    "Foley, Jason", "Díaz, Alexis", "Miller, Mason", "Jansen, Kenley", 
    "Estévez, Carlos", "Doval, Camilo", "Bednar, David", "Duran, Jhoan", 
    "Kimbrel, Craig", "Fairbanks, Pete", "Scott, Tanner", "Muñoz, Andrés"
]

# Filter the dataframe to only include these players
specific_df = df[df['last_name, first_name'].isin(specific_players)]

# Create a dataframe showing rank improvements for these specific players
specific_rank_changes = specific_df[['last_name, first_name', 'p_save', 'old', 'old_rank', 'new', 'new_rank', 'rank_improvement']]
specific_rank_changes = specific_rank_changes.sort_values(by='rank_improvement', ascending=False)

print("\nRank improvements for the 20 players with the most saves last season:")
display(specific_rank_changes)

# Also show players with the biggest rank improvements overall
biggest_improvers = df[['last_name, first_name', 'old', 'old_rank', 'new', 'new_rank', 'rank_improvement']]
biggest_improvers = biggest_improvers.sort_values(by='rank_improvement', ascending=False)

print("\nPlayers with biggest rank improvements from old to new scoring system:")
display(biggest_improvers.head(10))

Top 100 players by new scoring system with ranks:


,"last_name, first_name",player_id,old,old_rank,new,new_rank,p_save
209,"Skubal, Tarik",669373,1009.2,1,1075.2,1,0
520,"Wheeler, Zack",554430,970.3,2,1049.3,2,0
79,"Sale, Chris",519242,966.6,3,1021.6,3,0
778,"Helsley, Ryan",664854,744.1,15,969.1,4,49
152,"Clase, Emmanuel",661403,747.7,14,967.7,5,47
695,"Gilbert, Logan",669302,887.3,4,954.3,6,0
478,"Cease, Dylan",656302,867.8,5,915.8,7,0
252,"Ragans, Cole",666142,836.3,6,900.3,8,0
171,"Lugo, Seth",607625,779.8,7,845.8,9,0
229,"Burnes, Corbin",669203,777.5,9,842.5,10,0



Rank improvements for the 20 players with the most saves last season:


,"last_name, first_name",p_save,old,old_rank,new,new_rank,rank_improvement
629,"Foley, Jason",28,408.1,118,559.1,77,41
2,"Duran, Jhoan",23,437.3,102,569.3,71,31
345,"Finnegan, Kyle",38,510.5,75,675.5,44,31
758,"Jansen, Kenley",27,461.3,95,574.3,68,27
558,"Bednar, David",23,333.5,161,425.5,135,26
582,"Fairbanks, Pete",23,353.1,150,452.1,124,26
734,"Díaz, Alexis",28,396.3,124,517.3,100,24
571,"Scott, Tanner",22,567.4,57,711.4,35,22
742,"Suarez, Robert",36,583.6,55,737.6,33,22
457,"Yates, Kirby",33,667.1,31,833.1,12,19



Players with biggest rank improvements from old to new scoring system:


,"last_name, first_name",old,old_rank,new,new_rank,rank_improvement
32,"Harvey, Hunter",172.4,333,274.4,240,93
745,"Pérez, Cionel",146.0,363,239.0,278,85
367,"Rogers, Tyler",226.0,262,349.0,177,85
627,"Schreiber, John",184.4,313,271.4,244,69
348,"Cano, Yennier",281.3,202,426.3,134,68
244,"Poche, Colin",131.1,380,211.1,315,65
751,"Cimber, Adam",45.0,552,78.0,491,61
548,"Kahnle, Tommy",177.7,325,246.7,267,58
416,"Smith, Will",47.1,545,79.1,489,56
553,"Peralta, Wandy",87.7,455,126.7,404,51
